In [ ]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as go
datafile = 'youtube-new\CAvideos.csv'
#数据导入
data =pd.read_csv(datafile,encoding='utf-8')
data.info()
#将每个属性的描述性统计量展示
explore=data.describe(percentiles=[],include='all').T
#print('每个属性的描述性统计量展示:',explore)
#计算属性对应的空值个数
print('video_id的缺失值个数为:',data['video_id'].isnull().sum())
print('trending_date的缺失值个数为:',data['trending_date'].isnull().sum())
print('title的缺失值个数为:',data['title'].isnull().sum())
print('channel_title的缺失值个数为:',data['channel_title'].isnull().sum())
print('category_id的缺失值个数为:',data['category_id'].isnull().sum())
print('publish_time的缺失值个数为:',data['publish_time'].isnull().sum())
print('tags的缺失值个数为:',data['tags'].isnull().sum())
print('views的缺失值个数为:',np.sum(data['views']==0))
print('likes的缺失值个数为:',np.sum(data['likes']==0))
print('comment_count的缺失值个数为:',np.sum(data['comment_count']==0))
print('dislikes的缺失值个数为:',np.sum(data['dislikes']==0))
print('thumbnail_link的缺失值个数为:',data['thumbnail_link'].isnull().sum())
print('comments_disabled的缺失值个数为:',data['comments_disabled'].isnull().sum())
print('ratings_disabled的缺失值个数为:',data['ratings_disabled'].isnull().sum())
print('video_error_or_removed的缺失值个数为:',data['video_error_or_removed'].isnull().sum())
print('description的缺失值个数为:',data['description'].isnull().sum())
#求views属性的五数概括
import numpy as np
likes=data['likes']
likes = likes.sort_values()
likes_d = likes.values
min_likes = min(likes_d) #找出数组中的最小值
max_likes = max(likes_d) #找出数组中的最大值
Q1=np.percentile(likes_d,25)
Median=np.median(likes_d)
Q3=np.percentile(likes_d,75)
print(min_likes,Q1,Median,Q3,max_likes)
vdata = [
go.Box(
y=[min_likes, Q1, Median, Q3, max_likes] 
)
]
py.plot(vdata) # 绘制盒图
# 绘制直方图
import matplotlib.pyplot as plt
data0=data
any(data0.likes.isnull())
data0.dropna(subset=['likes'], inplace=True)
plt.hist(x = data0.likes, # 指定绘图数据
          bins = 100, # 指定直方图中条块的个数
          color = 'steelblue', # 指定直方图的填充色
          edgecolor = 'black' # 指定直方图的边框色
          )
# 添加x轴和y轴标签
plt.xlabel('likes')
plt.ylabel('frequency')
#plt.title('')
# 显示图形
plt.show()
sdata=data.sample(n=2000,axis=0)
#fig = go.Figure(go.Scatter(x=sdata.views, y=sdata.likes, text=sdata.title, mode='markers', name='2007'))
#fig.update_xaxes(title_text='views', type='log')
#fig.update_yaxes(title_text='likes')
#py.plot(fig, filename='pandas-multiple-scatter')
sdata0=sdata[~sdata['likes'].isin([0])]
slikes=sdata0['likes']
slikes = slikes.sort_values()
sdata['likes']=slikes
#直接删除
fig = go.Figure(go.Scatter(x=sdata.views, y=sdata.likes, text=sdata.title, mode='markers', name='2007'))
fig.update_xaxes(title_text='views', type='log')
fig.update_yaxes(title_text='likes')
py.plot(fig, filename='pandas-multiple-scatter')
#用均值替代缺失值
slikes=sdata['likes']
slikes = slikes.sort_values()
slikes_d = slikes.values
sMean=np.mean(slikes_d)
slikes=slikes.replace(0,sMean)
sdata1=sdata
sdata1['likes']=slikes
fig = go.Figure(go.Scatter(x=sdata1.views, y=sdata1.likes, text=sdata1.title, mode='markers', name='2007'))
fig.update_xaxes(title_text='views', type='log')
fig.update_yaxes(title_text='likes')
py.plot(fig, filename='pandas-multiple-scatter')
#利用KNN填补缺失值
from fancyimpute import KNN
slikes=sdata['likes']
slikes = slikes.sort_values()
fill_knn = KNN(k=3).fit_transform(sdata)
data = pd.DataFrame(fill_knn)
print(data.head())
#随机森林填充缺失值
from sklearn.ensemble import RandomForestRegressor
# 分成已知该特征和未知该特征两部分
known = sdata[sdata.c.notnull()].as_matrix()
uknown = sdata[sdata.c.isnull()].as_matrix()
# X为特征属性值
X = known[:, 1:3]
# print(X[0:10])
# Y为结果标签
y = known[:, 0]
print(y)
# 训练模型
rf = RandomForestRegressor(random_state=0, n_estimators=200, max_depth=3, n_jobs=-1)
rf.fit(X, y)
# 预测缺失值
predicted = rf.predict(uknown[:, 1:3])
print(predicted)
#将预测值填补原缺失值
sdata.loc[(sdata.c.isnull()), 'c'] = predicted